In [ ]:
# Importing libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from math import e,sqrt
from pyFTS.fcm import Activations
from pyFTS.models import hofts
from numpy.linalg import svd
from pyFTS.fcm import fts as fcm_fts
from pyFTS.partitioners import Grid
from pyFTS.common import Util
from pyFTS.common import Membership as mf
from scipy.optimize import least_squares
from scipy.optimize import leastsq
from pyFTS.benchmarks import Measures

In [ ]:
#defining FCM
class FuzzyCognitiveMap(object):
    def __init__(self, **kwargs):
        super(FuzzyCognitiveMap, self).__init__()
        self.order = kwargs.get('order',1)
        self.concepts = kwargs.get('partitioner',None)
        self.weights = [] 
        self.bias=[]
        self.activation_function = kwargs.get('activation_function', Activations.sigmoid) 
        #self.activation_function=softplus

    def activate(self, memberships):
        dot_products = np.zeros(len(self.concepts))
        for k in np.arange(0, self.order):
            dot_products += np.dot(np.array(memberships[k]).T, self.weights[k])+(self.bias[k])
        return self.activation_function( dot_products )


In [ ]:
def func(values, *arg):
    #print(values.shape)
    if len(values.shape) == 1:
      n_coefs = values.shape[0] + 1
      #return arg[0] + np.sum([values[i-1] * arg[i] for i in np.arange(1,n_coefs)])
      ret = 0
    else:
      n_coefs = values.shape[0] + 1
      n_inst = values.shape[1]
      ret = np.zeros(n_inst)
    for k in range(1, n_coefs):
      ret = ret + arg[k] * values[k-1]

    ret = ret + arg[0]

    return ret

In [ ]:

def func2(x, a, b, c):
    z1,z2 = x
    return a + (b * z1) + (c * z2)

def func3(x, a, b, c, d):
    z1,z2,z3 = x
    return a + (b * z1) + (c * z2) + (d * z3)

def func4(x, a, b, c, d, e):
    z1,z2,z3,z4 = x
    return a + (b * z1) + (c * z2) + (d * z3) + (e * z4)

# RHFCM Model
class FCM_FTS(hofts.HighOrderFTS):

    def __init__(self, **kwargs):
        super(FCM_FTS, self).__init__(**kwargs)

        num_concepts = self.partitioner.partitions

        self.num_fcms = kwargs.get('num_fcms', 2)

        self.fcm = []

        self.loss_function = kwargs.get('loss', func)

        for k in range(self.num_fcms):
          fcm_tmp = FuzzyCognitiveMap(**kwargs)
          weights = np.random.uniform(-1, 1, size=(self.order,num_concepts, num_concepts))
          specturalradius1=np.max(np.abs(np.linalg.eigvals(weights)))
          fcm_tmp.weights = weights*0.5/specturalradius1
          bias = np.random.uniform(-1, 1, size=(self.order,num_concepts))
          U,S,VT=svd(bias)
          specturalradius2=np.max(S)
          fcm_tmp.bias=bias*0.5/specturalradius2
          self.fcm.append(fcm_tmp) 
  

        # Coefficients
        self.theta = np.random.rand(self.num_fcms + 1)
        
    def forecast(self, data, **kwargs):
        y1 = []

        midpoints = np.array([fset.centroid for fset in self.partitioner])

        for t in np.arange(self.order, len(data)+1):

            sample = data[t - self.order : t]

            fuzzyfied = self.partitioner.fuzzyfy(sample, mode='vector')

            # Evaluate FCMs

            forecasts = []

            for fcm in self.fcm:
              activation=fcm.activate(fuzzyfied)
              forecasts.append(np.dot(midpoints, activation)/np.nanmax([1, np.sum(activation)]))

            # Combine the results

            #print(forecasts)

            result = self.loss_function(np.array(forecasts), *self.theta)

            if str(result) == 'nan' or result == np.nan or result == np.Inf:
               print('error')

            y1.append(result)

        return y1

    def run_fcm(self, fcm, data):
        ret = []
        midpoints = np.array([fset.centroid for fset in self.partitioner])
        for t in np.arange(self.order, len(data)+1):
            sample = data[t - self.order : t]
            fuzzyfied = self.partitioner.fuzzyfy(sample, mode='vector')
            activation = fcm.activate(fuzzyfied)
            final = np.dot(midpoints, activation)/np.nanmax([1, np.sum(activation)])
            ret.append(final)
        return ret

    def train(self, data, **kwargs):
        from scipy.optimize import curve_fit, least_squares, minimize, leastsq

        outputs = []

        for model in self.fcm:
          outputs.append(self.run_fcm(model, data)[:-1])

        f = lambda coef, y, x: self.loss_function(x, *coef) - y

        self.theta, flag = leastsq(f, x0 = self.theta, args=(data[self.order:], np.array(outputs))) 
        print(self.theta)

In [ ]:
# Malaysia load dataset
df=pd.read_csv('https://query.data.world/s/e5arbthdytod3m7wfcg7gmtluh3wa5', sep=';')

data = df['load'].values[:8760]
partitioner = Grid.GridPartitioner(data=data, npart=5, mf=mf.trimf)
y= data

rmse = []
mape = []
u = []

for experiment in range(0, 20):
  from pyFTS.common import Util
  for  count,train, test in Util.sliding_window(data, windowsize=2000, train_rate=0.9, increment_rate=0.2):
     model = FCM_FTS(partitioner=partitioner, order=2, num_fcms=2, 
                  activation_function=softplus,
                  loss=func)
     model.fit(train)
     _rmse,_mape,_u= Measures.get_point_statistics(test, model)
     rmse.append(_rmse)
     mape.append(_mape)
     u.append(_u)
     

print(np.mean(rmse))
print(np.mean(mape))
print(np.mean(u))
print(np.min(rmse))
print(np.min(mape))
print(np.min(u))
print(np.max(rmse))
print(np.max(mape))
print(np.max(u))


[ 2.09455713e+05  3.94213108e+00 -7.91884049e+00]
[ 2.02322523e+06 -2.78771333e+01 -1.51977218e+01]
[ 8.57775444e+05 -1.10124007e+01 -8.36953162e+00]
[-4.52140936e+05  1.00151609e+01  1.97543393e+00]
[ 3.92976103e+05 -5.97774452e-01 -7.93204412e+00]
[-5.68389243e+05  3.83174334e+00  1.06199803e+01]
[-2.76178828e+05  6.81850080e+00  6.74306093e-01]
[ 4.93398129e+05 -7.96274748e+00 -2.83189814e+00]
[ 3.91570926e+05 -4.93043926e+00 -2.87321278e+00]
[-7.57756589e+05  5.97854521e+00  1.18207558e+01]
[ 2.53738742e+05 -1.11708361e+01  6.28835430e+00]
[-2.14222425e+05 -5.54171409e+00  1.07507992e+01]
[ 4.29278591e+05 -3.14466408e+00 -5.54702331e+00]
[ 8.70163386e+04  2.89045638e+00 -3.93438196e+00]
[ 5.36637952e+05 -1.60066315e+01  5.17321385e+00]
[ 6.75604968e+05 -7.17352620e+00 -7.77302445e+00]
[ 2.94662703e+06 -3.85316546e+01 -2.70424928e+01]
[-3.83142075e+05  1.16536129e+01 -1.36482403e+00]
[ 1.55665682e+06  3.97174576e+00 -3.87302282e+01]
[-2.61892607e+05 -4.32757175e+00  1.28689602e+01]
